# M2 | Exploration Notebook

In this notebook, you will do a first exploration of the data set that you will use for your project. One part of this exploration is guided, i.e. we will ask you to solve specific questions (task 1-3). The other part is open, i.e. we will ask you to come up with your own exploration ideas (task 4). 

Please upload your solved notebook to Moodle (under Milestone 2 Submission) adding your SCIPER number in title, example: m2-calcularis-456392.ipynb


## Brief overview of Calcularis
[Calcularis](https://school.alemira.com/de/calcularis/) by Alemira School is a mathematics learning program developed with neuroscientists and computer scientists from ETH Zurich. It promotes the development and interaction of the different areas of the brain that are responsible for processing numbers and quantities and solving mathematical tasks. Calcularis can be used from 1st grade to high school. Children with dyscalculia also benefit in the long term and overcome their arithmetic weakness.


For the guided part of the exploration we will focus on the three main tables:
* ***users***: meta information about users (i.e. total time spent learning with Calcularis, geographic location).
* ***events***: events done by the users in the platform (i.e. playing a game, selecting a new animal in the zoo simulation).
* ***subtasks***: sub-tasks with answer attempts solved by users, primarily in the context of game events.

### Users
* **user_id**: unique identifier of user in database.
* **learning_time_ms**: total learning time in milliseconds per user, totaled over all events. This is calculated by subtracting non-learning time (i.e. if the user is idle for more than one minute).
* **start**: timestamp of initial login.
* **end**: timestamp of last log-off.
* **logged_in_time_ms**: total time between login and log-off, in milliseconds. This will always be larger than learning_time_ms.
* **language**: code for language the user is using Calcularis in (i.e. *fr* for French, *de* for german).
* **country**: code for the country the user is using Calcularis from (i.e. *CH* for Switzerland).

### Events
* **event_id**: unique identifier of event in database.
* **user_id**: user who peformed the event.
* **type**: type of event (task, help, game_start_pause, training_selector, shop, attention_booster, attention_booster_over).
* **mode**: only valid for events of type 'task', representing type of game play (normal, end_of_nr: end of number range, free_training, repetition, favorite, practice).
* **game_name**: name of the game the user is playing. Calcularis has 29 games.
* **learning_time_ms**: total learning time over a single game (event). To see aggregated learning time, examine learning_time_ms in the users table.
* **number_range**: the number range of the exercise (0-10, 0-20, ... 0-1000). The larger the number range, the more difficult the exercise variation.
* **start**: start timestamp of the event.
* **end**: end timestamp of the event.
* **skill_id**: skill number that this game (event) corresponds to.

### Subtasks
*There are many columns in subtasks corresponding to fields present for specific games in Calcularis. More details about corresponding fields for subtasks can be found [here](https://drive.google.com/file/d/1C7bKfHDdKnDzr9-EqWniYoR35ro2QuBd/view?usp=sharing). The fields discussed below are present for the majority of subtasks.*

* **subtask_id**: unique identifier of the subtask in the database.
* **event_id**: unique identifier of event in database.
* **user_id**: user who peformed the event.
* **answer**: the user's answer to the subtask.
* **correct**: `True` if user obtained the correct value in their answer attempt.
* **correctAnswerObject**: the correct answer to the question.
* **hasProperResult**: `True` if this is a question that can be answered by a student. `False` if it corresponds to the SecretNumber game tasks, which consists of counting attempts.
* **range**: number_range for the task (i.e. 0-20 described as R20).
* **subtask_finished_timestamp**: ending timestamp for this subtask (within a specific event).
* **type**: type of subtask event like ConciseEquationTaskDescription, ConciseOrderingTaskDescription (mostly corresponding to individual games, as detailed in the *Subtask Representations* document below).

### Useful Metadata Files
- [Subtask Representations](https://drive.google.com/file/d/1C7bKfHDdKnDzr9-EqWniYoR35ro2QuBd/view?usp=sharing)
- [Number Representations](https://docs.google.com/document/d/1iD60mydGXjB-6HBQakAKI_GGfQaBBQkWcg5eQiAch_A/edit?usp=sharing)
- [Ideas for Calcularis analysis from the Alemira School](https://docs.google.com/document/d/1vM0Gz4f0jStp_wuhBYvnP-VjaBAxzQpVzXomaDfq0D8/edit?usp=sharing)
- [Skill map](https://drive.google.com/file/d/1yJTosk1RV1qzHb-AG_-FLdjlphBsYzoq/view?usp=sharing) representing the relationships between exercises in Calcularis. If you would like to dive deeper into the skill network (programatically), you can use this [.dot file](https://docs.google.com/document/d/1XDPkq7LaAQ3-lAhAvDTLXuiq5u8EKJME/edit?usp=share_link&ouid=104052806655845468726&rtpof=true&sd=true) representing the graph and the corresponding [labels](https://drive.google.com/file/d/1Pkbiz-_bzK80q2l3mkBS_K4wU9sSCY-R/view?usp=sharing).

In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = 'new_data_1000' #You many change the directory

users = pd.read_csv('{}/calcularis_small_users.csv'.format(DATA_DIR), index_col=0)
events = pd.read_csv('{}/calcularis_small_events.csv'.format(DATA_DIR), index_col=0)
subtasks = pd.read_csv('{}/calcularis_small_subtasks.csv'.format(DATA_DIR), index_col=0)

In [2]:
users.head(5)

,learning_time_ms,start,end,logged_in_time_ms,language,country
user_id,,,,,,
1,14032710,2022-11-02T08:37:56.549Z,2023-02-09T11:08:02.599Z,22151340,de,CH
2,16268350,2022-09-07T07:53:38.865Z,2023-02-09T08:39:14.692Z,85421273,nl,NL
3,8012030,2021-09-27T07:45:51.806Z,2022-01-13T12:14:09.565Z,16651482,de,DE
4,1414421,2019-11-12T12:18:15.724Z,2020-10-02T09:20:28.798Z,4561768,de,CH
5,17502108,2022-04-26T11:38:44.114Z,2022-08-29T15:52:11.087Z,25601470,de,CH


In [3]:
events.head(5)

,user_id,mode,game_name,learning_time_ms,number_range,start,end,skill_id,type
event_id,,,,,,,,,
0,1,NORMAL,Subitizing,8835.0,R10,2022-11-02T08:39:12.355Z,2022-11-02T08:39:25.130Z,1.0,task
1,1,NORMAL,Conversion,21167.0,R10,2022-11-11T10:26:27.893Z,2022-11-11T10:26:49.260Z,4.0,task
2,1,NORMAL,Conversion,11182.0,R10,2022-11-18T10:34:01.044Z,2022-11-18T10:34:12.423Z,7.0,task
3,1,NORMAL,Landing,6823.0,R10,2022-11-25T10:32:43.428Z,2022-11-25T10:32:56.986Z,19.0,task
4,1,END_OF_NR,Conversion,9107.0,R10,2022-12-02T10:44:40.555Z,2022-12-02T10:44:49.874Z,7.0,task


In [4]:
subtasks.head(5)

,event_id,user_id,aim,answer,answerMode,availableNumbers,correct,correctAnswerObject,correctNumber,destination,...,startPosition,subtask_finished_timestamp,target,timeoutInSeconds,timeoutInSteps,type,upperBound,divisor,orderIndependent,step
subtask_id,,,,,,,,,,,,,,,,,,,,,
0,0,1,NaN,4,NaN,NaN,True,4,4.0,NaN,...,NaN,2022-11-02T08:39:24.930Z,NaN,NaN,NaN,ConciseSubitizingTaskDescription,NaN,NaN,NaN,NaN
1,0,1,NaN,1,NaN,NaN,True,NaN,NaN,NaN,...,NaN,2022-11-02T08:39:24.930Z,NaN,0.0,2.0,ConciseTimeoutDescription,NaN,NaN,NaN,NaN
2,1,1,NaN,3,NaN,NaN,True,3,NaN,ARABIC,...,NaN,2022-11-11T10:26:49.007Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
3,2,1,NaN,5,NaN,NaN,True,5,NaN,EXPLODED_BEAM,...,NaN,2022-11-18T10:34:12.191Z,NaN,NaN,NaN,ConciseConversionTaskDescription,NaN,NaN,NaN,NaN
4,3,1,3.0,"{'a': 2, 'b': 2.0402703}",NaN,NaN,False,"{'a': 3, 'b': 3.0}",NaN,NaN,...,0.5,2022-11-25T10:32:56.805Z,NaN,NaN,NaN,ConciseLandingTaskDescription,3.5,NaN,NaN,NaN


## Task 1: Simple Statistics

In this task you are asked to do a first coarse exploration of the data set, using simple statistics and visualizations.

#### a) How many distinct participants do we have in the data set?


In [ ]:
## Your code goes here

#### b) How many games were played per user? Please provide a visualization and discuss the distribution.

In [ ]:
# Your code goes here

*Your discussion/interpretation goes here*


#### c) What are the 10 most popular games played (events)? Please provide visualizations of the frequency of the top 10 games.

Hint: See `game_name` in table `events`.

In [ ]:
# Your code goes here

*Your discussion/interpretation goes here*

## Task 2: Static Analysis

In this second task, you will do a univariate an multivariate exploration of some aggregated features.

#### a) Build a data frame containing one row per user:

``[user_id, ui_locale, num_events, num_games, percentage_correct, total_not_learning_time_ms]``


The features are defined as follows:

- **ui_locale**: geographic region the student is from

- **num_events**: total number of events of the student  

- **num_games**: total number of unique games played by the student

- **percentage_correct**: number of correct answers/total number of answers (hint: see `correct` in subtask table).

- **total_not_learning_time_ms**: total amount of ms not spent actively learning

In [ ]:
# Your code for building the data frame here

b) Perform a univariate analysis (including descriptive statistics and visualizations) for the five features (`ui_locale, num_events, num_games, percentage_correct, total_not_learning_time_ms`) of your dataframe. Please check the lecture slides regarding information on how to perform a univariate analysis for categorical and numerical features. Discuss your results: how are the features distributed? Are there any anomalities?

In [ ]:
# Your code for univariate analysis here

*Your discussion/interpretation goes here*

c) Come up with two additional features on your own and add them to the dataframe. Please provide an explanation/description of your features as well as an argument/hypothesis of why you think these features are interesting to explore.

In [ ]:
# Your code for computing the features and adding them to the df goes here

*Your feature descriptions and arguments/hypotheses go here*

d) Perform a univariate analysis of your features (including descriptive statistics and visualization). What can you observe? Do the results confirm your hypotheses?

In [ ]:
# Your code for univariate analysis goes here

*Your discussion/interpretation goes here*

e) Perform a multivariate analysis for two pairs of features of your choice. Please provide a metric and a visualization for both pairs. Please discuss: why did you choose these two pairs? What was your hypothesis? Do the results confirm your hypothesis?

In [ ]:
# Your code for multivariate analysis goes here

*Your discussion/interpretation goes here*

# Task 3: Time-Series Analysis

In the last task, you will perform a time-series analysis.


#### a) Build a data frame containing one row per user per week:

``[user_id, week, num_events, num_questions, percentage_correct, num_shop]``


The features are defined as follows:

- **num_questions**: total number of questions the student answered **per week**.

- **num_events**: total number of events of the student per week.

- **percentage_correct**: number of correct answers/total number of answers  **per week** (hint: answer = correct). If desired, you may assign a weight of 0.5 to partially correct answers.

- **num_shop**: total number of shop events **per week** (where the student wins animals for their zoo with rewards from the games).

Where week 0 is the first week the specific user solved a task in the platform, i.e., the user's earliest entry in the subtasks table.

Hint: You may extract the week of the year (dt.week) from the timestamps. 

You can limit the number of weeks to 13, i.e. for each user we just look at the first 13 weeks of data.
Yoy may change and justify your choice for the number of weeks.

In [ ]:
# Your code for building the dataframe goes here

#### b) Select two features and analyze their behavior over time. Please provide a hypothesis and visualization for both features. For ideas on how to perform a time series exploration, please check the lecture slides and notebook. Discuss your results: what do you observe? Do the results confirm your hypotheses?

*Your discussion goes here*

# Task 4: Creative extension 

Please provide **one** new hypothesis you would like to explore with the data and provide a visualization for it. Discuss your results: what do you observe? Do the results confirm your hypotheses?



In [ ]:
# Your creative visualization here

*Your discussion goes here*